In [ ]:
import pandas
import numpy

In [ ]:
open_payments = pandas.read_csv('../../data/processed/open_payments.csv', index_col=0)

In [ ]:
all_doctors = pandas.read_csv('../../data/processed/all.csv', index_col=0)

In [ ]:
apply_func = lambda x: x.upper() if isinstance(x, str) else None

open_payments['physician_first_name'] = open_payments['physician_first_name'].apply(apply_func)
open_payments['physician_last_name' ] = open_payments['physician_last_name' ].apply(apply_func)

all_doctors['first_name'] = all_doctors['first_name'].apply(apply_func)
all_doctors['last_name' ] = all_doctors['last_name' ].apply(apply_func)

In [ ]:
all_doctors = all_doctors\
    .groupby(['first_name', 'last_name', 'state'])\
    .first()\
    .reset_index()

In [ ]:
merged = pandas.merge(
    open_payments, 
    all_doctors.reset_index(), # preserve individual doctors in column `index`
    how='right',
    left_on=['physician_first_name', 'physician_last_name'],#, 'specialty_code'],
    right_on=['first_name', 'last_name'],#, 'specialty_code']
    suffixes=['_open_payments', None],
)

In [ ]:
merged

In [ ]:
pandas.merge(
    open_payments[open_payments['applicable_manufacturer_or_applicable_gpo_making_payment_id'] == 100000131389], 
    all_doctors.reset_index(), # preserve individual doctors in column `index`
    how='inner',
    left_on=['physician_first_name', 'physician_last_name'],#, 'specialty_code'],
    right_on=['first_name', 'last_name'],#, 'specialty_code']
    suffixes=['_open_payments', None],
)

In [ ]:
horizon = merged[merged['applicable_manufacturer_or_applicable_gpo_making_payment_id'] == 100000131389]

In [ ]:
horizon.groupby('physician_profile_id').first()

In [ ]:

# how many unique doctors take money?
for source in all_doctors['src'].unique():
    individual_doctors_taking_money = horizon.groupby('physician_profile_id').first()
    likely_prescribers_count = len(
        all_doctors[(all_doctors['src'] == source)]
    )
    on_take = len(
        individual_doctors_taking_money[
            (individual_doctors_taking_money['src'] == source)
            
        ]
    )
    print(source, on_take)
    on_take_pct = on_take / likely_prescribers_count
    print(f"For data from {source}, % of doctors on take is {round(on_take_pct * 100, 5)}% out of {likely_prescribers_count}")
"""For data from asoprs, % of doctors on take is 42.28769% out of 577
For data from endocrinologists, % of doctors on take is 18.10437% out of 939
For data from tepezza, % of doctors on take is 65.74468% out of 470"""

In [ ]:
merged.to_csv('../../data/processed/all_transactions.csv')

In [ ]:
total_money = merged\
    .groupby('physician_profile_id')\
    .sum()\
    ['total_amount_of_payment_usdollars']

In [ ]:
dates = merged\
    .groupby('physician_profile_id')\
    ['date_of_payment']\
    .apply(','.join)
    

In [ ]:
# get grouped paid doctors and all unpaid doctors as one
agg = merged\
    .groupby('physician_profile_id')\
    .first()
agg['total_money'] = total_money
agg['dates_of_payments'] = dates
agg = agg\
    .reset_index(drop=True)\
    .append(
        merged[pandas.isna(merged['physician_profile_id'])]
    )\
    .drop(columns=['index', 'total_amount_of_payment_usdollars', 'date_of_payment'])\
    .reset_index(drop=True)

In [ ]:
agg

In [ ]:
agg.to_csv('../../data/processed/aggregated.csv')